In [97]:
# Import the necessary libraries
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from newspaper import Article
import syllables

In [98]:
# Create the output dataframe
output_data = pd.DataFrame(columns=["URL_ID", "URL", "POSITIVE SCORE", "NEGATIVE SCORE", "POLARITY SCORE", "SUBJECTIVITY SCORE",
                                    "AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS", "FOG INDEX",
                                    "AVG NUMBER OF WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT",
                                    "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"])


In [99]:
# Read URLs from Excel file
input_data = pd.read_excel('/content/Input.xlsx')

In [100]:
# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [102]:
# Iterate over the URLs
for i, row in input_data.iterrows():
    url = row['URL']
    
    try:
        # Extract article text
        article = Article(url)
        article.download()
        article.parse()
        article_text = article.text
        
        # Perform sentiment analysis
        sentiment_scores = sia.polarity_scores(article_text)
        
        # Compute other variables
        word_count = len(nltk.word_tokenize(article_text))
        sentence_count = len(nltk.sent_tokenize(article_text))
        avg_sentence_length = word_count / sentence_count
        
        # Compute percentage of complex words (example, using a threshold of word length > 5)
        words = nltk.word_tokenize(article_text)
        complex_word_count = sum(1 for word in words if len(word) > 5)
        percentage_complex_words = (complex_word_count / word_count) * 100
        
        # Compute other variables and scores
        fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
        avg_words_per_sentence = word_count / sentence_count
        syllables_per_word = sum(syllables.estimate(word) for word in words) / word_count
        personal_pronouns = sum(1 for word in words if word.lower() in ["i", "me", "my", "mine"])
        avg_word_length = sum(len(word) for word in words) / word_count
        
        # Create a new row for the output dataframe
        output_row = {
            "URL_ID": i+1,
            "URL": url,
            "POSITIVE SCORE": sentiment_scores["pos"],
            "NEGATIVE SCORE": sentiment_scores["neg"],
            "POLARITY SCORE": sentiment_scores["compound"],
            "SUBJECTIVITY SCORE": sentiment_scores["compound"],
            "AVG SENTENCE LENGTH": avg_sentence_length,
            "PERCENTAGE OF COMPLEX WORDS": percentage_complex_words,
            "FOG INDEX": fog_index,
            "AVG NUMBER OF WORDS PER SENTENCE": avg_words_per_sentence,
            "COMPLEX WORD COUNT": complex_word_count,
            "WORD COUNT": word_count,
            "SYLLABLE PER WORD": syllables_per_word,
            "PERSONAL PRONOUNS": personal_pronouns,
            "AVG WORD LENGTH": avg_word_length
        }
        
        # Append the row to the output dataframe
        output_data = pd.concat([output_data, pd.DataFrame([output_row])], ignore_index=True)
        
    except Exception as e:
        print(f"Error processing URL: {url}")
        print(e)
        continue

Error processing URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ on URL https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error processing URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ on URL https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Error processing URL: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/ on URL https://

In [104]:
# Save the output dataframe as a CSV file
output_file_path = "/content/output.csv"
output_data.to_csv(output_file_path, index=False)
print("Output CSV file saved successfully.")

Output CSV file saved successfully.
